In [1]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
import re
import pandas as pd
from gensim import models
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Elise\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Elise\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Elise\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [17]:
model_fox = Word2Vec.load("./models/fox.model")
model_reuters = Word2Vec.load("./models/reuters.model")
model_the_hill = Word2Vec.load("./models/the_hill.model")
model_cnn = Word2Vec.load("./models/cnn.model")
model_nyt = Word2Vec.load("./models/nyt.model")

In [18]:
adjectives = ['disorganized', 'devious', 'impressionable', 'circumspect', 'impassive', 
             'aimless', 'effeminate', 'unfathomable', 'fickle', 'unprincipled', 'inoffensive', 
             'reactive', 'providential', 'resentful', 'bizarre', 'impractical',
             'sarcastic', 'misguided', 'imitative', 'pedantic', 'venomous', 'erratic', 'insecure', 
             'resourceful', 'neurotic', 'forgiving', 'profligate', 'whimsical', 'assertive', 
             'incorruptible', 'individualistic', 'faithless', 'disconcerting', 'barbaric', 
             'hypnotic', 'vindictive', 'observant', 'dissolute', 'frightening', 'complacent', 
             'boisterous', 'pretentious', 'disobedient', 'tasteless', 'sedentary', 
             'sophisticated', 'regimental', 'mellow', 'deceitful', 'impulsive', 'playful', 
             'sociable', 'methodical', 'willful', 'idealistic', 'boyish', 'callous', 'pompous', 
             'unchanging', 'crafty', 'punctual', 'compassionate', 'intolerant', 'challenging', 
             'scornful', 'possessive', 'conceited', 'imprudent', 'dutiful', 'lovable', 
             'disloyal', 'dreamy', 'appreciative', 'forgetful', 'unrestrained', 'forceful', 
             'submissive', 'predatory', 'fanatical', 'illogical', 'tidy', 'aspiring', 'studious', 
             'adaptable', 'conciliatory', 'artful', 'thoughtless', 'deceptive', 'frugal', 
             'reflective', 'insulting', 'unreliable', 'stoic', 'hysterical', 'rustic', 
             'inhibited', 'outspoken', 'unhealthy', 'ascetic', 'skeptical', 'painstaking', 
             'contemplative', 'leisurely', 'sly', 'mannered', 'outrageous', 'lyrical', 
             'placid', 'cynical', 'irresponsible', 'vulnerable', 'arrogant', 'persuasive', 
             'perverse', 'steadfast', 'crisp', 'envious', 'naive', 'greedy', 'presumptuous', 
             'obnoxious', 'irritable', 'dishonest', 'discreet', 'sporting', 'hateful', 
             'ungrateful', 'frivolous', 'reactionary', 'skillful', 'cowardly', 'sordid', 
             'adventurous', 'dogmatic', 'intuitive', 'bland', 'indulgent', 'discontented', 
             'dominating', 'articulate', 'fanciful', 'discouraging', 'treacherous', 
             'repressed', 'moody', 'sensual', 'unfriendly', 'optimistic', 'clumsy', 
             'contemptible', 'focused', 'haughty', 'morbid', 'disorderly', 'considerate', 
             'humorous', 'preoccupied', 'airy', 'impersonal', 'cultured', 'trusting', 
             'respectful', 'scrupulous', 'scholarly', 'superstitious', 'tolerant', 
             'realistic', 'malicious', 'irrational', 'sane', 'colorless', 'masculine', 
             'witty', 'inert', 'prejudiced', 'fraudulent', 'blunt', 'childish', 'brittle', 
             'disciplined', 'responsive', 'courageous', 'bewildered', 'courteous', 
             'stubborn', 'aloof', 'sentimental', 'athletic', 'extravagant', 'brutal', 
             'manly', 'cooperative', 'unstable', 'youthful', 'timid', 'amiable', 'retiring', 
             'fiery', 'confidential', 'relaxed', 'imaginative', 'mystical', 'shrewd', 
             'conscientious', 'monstrous', 'grim', 'questioning', 'lazy', 'dynamic', 
             'gloomy', 'troublesome', 'abrupt', 'eloquent', 'dignified', 'hearty', 'gallant', 
             'benevolent', 'maternal', 'paternal', 'patriotic', 'aggressive', 'competitive', 
             'elegant', 'flexible', 'gracious', 'energetic', 'tough', 'contradictory', 
             'shy', 'careless', 'cautious', 'polished', 'sage', 'tense', 'caring', 
             'suspicious', 'sober', 'neat', 'transparent', 'disturbing', 'passionate', 
             'obedient', 'crazy', 'restrained', 'fearful', 'daring', 'prudent', 'demanding', 
             'impatient', 'cerebral', 'calculating', 'amusing', 'honorable', 'casual',
             'sharing', 'selfish', 'ruined', 'spontaneous', 'admirable', 'conventional', 
             'cheerful', 'solitary', 'upright', 'stiff', 'enthusiastic', 'petty', 'dirty', 
             'subjective', 'heroic', 'stupid', 'modest', 'impressive', 'orderly', 'ambitious', 
             'protective', 'silly', 'alert', 'destructive', 'exciting', 'crude', 'ridiculous', 
             'subtle', 'mature', 'creative', 'coarse', 'passive', 'oppressed', 'accessible', 
             'charming', 'clever', 'decent', 'miserable', 'superficial', 'shallow', 'stern', 
             'winning', 'balanced', 'emotional', 'rigid', 'invisible', 'desperate', 'cruel', 
             'romantic', 'agreeable', 'hurried', 'sympathetic', 'solemn', 'systematic', 
             'vague', 'peaceful', 'humble', 'dull', 'expedient', 'loyal', 'decisive', 
             'arbitrary', 'earnest', 'confident', 'conservative', 'foolish', 'moderate', 
             'helpful', 'delicate', 'gentle', 'dedicated', 'hostile', 'generous', 'reliable', 
             'dramatic', 'precise', 'calm', 'healthy', 'attractive', 'artificial', 
             'progressive', 'odd', 'confused', 'rational', 'brilliant', 'intense', 
             'genuine', 'mistaken', 'driving', 'stable', 'objective', 'sensitive', 
             'neutral', 'strict', 'angry', 'profound', 'smooth', 'ignorant', 'thorough', 
             'logical', 'intelligent', 'extraordinary', 'experimental', 'steady', 
             'formal', 'faithful', 'curious', 'reserved', 'honest', 'busy', 'educated', 
             'liberal', 'friendly', 'efficient', 'sweet', 'surprising', 'mechanical', 
             'clean', 'critical', 'criminal', 'soft', 'proud', 'quiet', 'weak', 'anxious', 
             'solid', 'complex', 'grand', 'warm', 'slow', 'false', 'extreme', 'narrow', 
             'dependent', 'wise', 'organized', 'pure', 'directed', 'dry', 'obvious', 'popular', 
             'capable', 'secure', 'active', 'independent', 'ordinary', 'fixed', 'practical', 
             'serious', 'fair', 'understanding', 'constant', 'cold', 'responsible', 'deep', 
             'religious', 'private', 'simple', 'physical', 'original', 'working', 'strong', 
             'modern', 'determined', 'open', 'political', 'difficult', 'knowledge', 'kind']

In [38]:
testing_words = ["gun", "metoo", "america", "tax", "abortion", "democrat", "republican", "nra", 
                 "trump", "biden", "china", "gay", "lesbian"]

women_list = ['daughter', 'mother', 'woman', 'girl', 'female', 
              'sister', 'aunt', 'niece']
men_list = ['son', 'father', 'man', 'boy', 'male', 'brother',
            'uncle', 'nephew']

straight_list = ['heterosexual', 'straight']
lgbt_list = ['gay', 'trans', 'bisexual', 'lesbian', 'homosexual', 'transgender']

religion_list = ['god', 'church', 'temple', 'holy', 'synagogue', 'religion', 'sacred', 'jesus', 'worship']
secular_list = ['secular', 'atheist', 'agnostic']

d_list = ['democrat', 'liberal', 'right', 'pelosi', 'obama']
r_list = ['republican', 'conservative', 'left', 'trump', 'mcconnell']

school_subjects = ['english', 'math', 'art', 'science', 'history', 'music', 'geography', 'gym', 'drama', 'biology',
                   'chemistry', 'physics', 'technology', 'foreign languages', 'social studies', 'philosophy', 'design',
                   'literature', 'algebra', 'geometry']

#careers = 

In [28]:
def print_adj_dists(news_source, model, word_list, compare_wordlist):    
    for word in word_list:
      word_dists = []
      for ind in range(len(compare_wordlist)):
        adj = compare_wordlist[ind]
        try:
          dist = model.wv.distance(word, adj)
          word_dists.append([news_source, word, adj, dist])
        except:
          word_dists.append([news_source, word, adj, None])

      df = pd.DataFrame(word_dists, columns=['source','word', 'adj', 'dist'])

      print(df.shape)

      df_sorted = df.sort_values(by='dist', ascending=True)
      print(df_sorted.head(5))
        
def print_word_dists_each_model(wordlist, compare_wordlist):
    print_adj_dists("Fox News", model_fox, wordlist, compare_wordlist)
    print_adj_dists("Reuters", model_reuters, wordlist, compare_wordlist)
    print_adj_dists("The Hill", model_the_hill, wordlist, compare_wordlist)
    print_adj_dists("NYT", model_nyt, wordlist, compare_wordlist)
    print_adj_dists("CNN", model_cnn, wordlist, compare_wordlist)
    
def find_avg_loc(model, wordlist):
    avg = [0]*100
    for i in range(0, len(wordlist)):
        avg += model.wv.get_vector(wordlist[i])
    avg /= len(wordlist)
    return avg

def find_avg_dists(news_source, model, wordlist_name, wordlist, compare_wordlist, print_output=True):
    word_dists = []
    avg_loc = find_avg_loc(model, wordlist)
    for ind in range(len(compare_wordlist)):
      compare_word = compare_wordlist[ind]
      try: 
        dist = abs(np.linalg.norm(avg_loc-model.wv.get_vector(compare_word)))
        word_dists.append([news_source, wordlist_name, compare_word, dist])
      except:
        word_dists.append([news_source, wordlist_name, compare_word, None])

    df = pd.DataFrame(word_dists, columns=['news_source', 'wordlist_name', 'compare_word', 'dist'])
    
    if print_output:
        df_sorted = df.sort_values(by='dist', ascending=True)
        print(df.shape)
        print(df_sorted.head())
    return df

def find_avg_dists_each_model(wordlist_name, wordlist, compare_wordlist):
    find_avg_dists("Fox", model_fox, wordlist_name, wordlist, compare_wordlist)
    #find_avg_dists("Reuters", model_reuters, wordlist_name, wordlist, compare_wordlist)
    #find_avg_dists("The Hill", model_the_hill, wordlist_name, wordlist, compare_wordlist)
    #find_avg_dists("NYT", model_nyt, wordlist_name, wordlist, compare_wordlist)
    #find_avg_dists("CNN", model_cnn, wordlist_name, wordlist, compare_wordlist)
    
def find_avg_dists_compare(news_source, model, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist):
    dists1 = find_avg_dists(news_source, model, wordlist_name1, wordlist1, compare_wordlist, print_output=False)
    dists2 = find_avg_dists(news_source, model, wordlist_name2, wordlist2, compare_wordlist, print_output=False)

    print("""\nif difference between dists is positive, association is stronger 
with wordlist1. if negative, association is stronger with wordlist2\n""")
    
    diffs = pd.DataFrame()
    diffs['news_source'] = dists1['news_source']
    diffs['wordlist1'] = wordlist_name1
    diffs['wordlist2'] = wordlist_name2
    diffs['compare_word'] = dists1['compare_word']
    diffs['diff'] = dists1['dist']-dists2['dist']
    
    df_sorted = diffs.sort_values(by='diff', ascending=False)
    print(df_sorted.head())
    if len(df_sorted) > 5:
        df_sorted = diffs.sort_values(by='diff', ascending=True)
        print(df_sorted.head())
        
def find_avg_dists_compare_each_model(wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist):
    find_avg_dists_compare("Fox News", model_fox, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_dists_compare("Reuters", model_reuters, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_dists_compare("The Hill", model_the_hill, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_dists_compare("NYT", model_nyt, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_dists_compare("CNN", model_cnn, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)  
        
def find_avg_relative_dists(news_source, model, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist):
    dists1 = find_avg_dists(news_source, model, wordlist_name1, wordlist1, compare_wordlist, print_output=False)
    dists2 = find_avg_dists(news_source, model, wordlist_name2, wordlist2, compare_wordlist, print_output=False)
    
    rel1 = dists1['dist']/(dists1['dist']+dists2['dist'])
    rel2 = dists2['dist']/(dists1['dist']+dists2['dist'])
    
    rels = pd.DataFrame()
    rels['news_source'] = dists1['news_source']
    rels['compare_word'] = dists1['compare_word']
    rels['wordlist1'] = wordlist_name1
    rels['d1/d1+d2'] = rel1
    rels['wordlist2'] = wordlist_name2
    rels['d2/d1+d2'] = rel2
    rels['compare_word'] = dists1['compare_word']
    
    print(rels)
    
def find_rel_dists_compare_each_model(wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist):
    find_avg_relative_dists("Fox News", model_fox, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_relative_dists("Reuters", model_reuters, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_relative_dists("The Hill", model_the_hill, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_relative_dists("NYT", model_nyt, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)
    find_avg_relative_dists("CNN", model_cnn, wordlist_name1, wordlist1, wordlist_name2, wordlist2, compare_wordlist)  


In [15]:
find_avg_dists_compare_each_model("r", religion_list, "s", secular_list, adjectives)


if difference between dists is positive, association is stronger 
with wordlist1. if negative, association is stronger with wordlist2

    news_source wordlist1 wordlist2  compare_word      diff
365    Fox News         r         s       liberal  0.940206
318    Fox News         r         s  conservative  0.825804
334    Fox News         r         s   progressive  0.812022
320    Fox News         r         s      moderate  0.766365
87     Fox News         r         s     deceptive  0.742832
    news_source wordlist1 wordlist2 compare_word      diff
359    Fox News         r         s     faithful -0.265766
20     Fox News         r         s     venomous -0.225071
380    Fox News         r         s      complex -0.201097
268    Fox News         r         s       heroic -0.167153
236    Fox News         r         s        sober -0.142112


In [32]:
find_rel_dists_compare_each_model("woman", women_list, "man", men_list, ["submissive"])

  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0    Fox News   submissive     woman  0.521833       man  0.478167
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0     Reuters   submissive     woman  0.516779       man  0.483221
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0    The Hill   submissive     woman  0.526956       man  0.473044
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0         NYT   submissive     woman  0.505395       man  0.494605
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0         CNN   submissive     woman   0.51115       man   0.48885


In [39]:
find_rel_dists_compare_each_model("lgbtq", lgbt_list, "straight", straight_list, ["love", "family", "america"])

  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0    Fox News         love     lgbtq  0.510260  straight  0.489740
1    Fox News       family     lgbtq  0.492736  straight  0.507264
2    Fox News      america     lgbtq  0.510355  straight  0.489645
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0     Reuters         love     lgbtq  0.516145  straight  0.483855
1     Reuters       family     lgbtq  0.515259  straight  0.484741
2     Reuters      america     lgbtq  0.509871  straight  0.490129
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0    The Hill         love     lgbtq  0.546670  straight  0.453330
1    The Hill       family     lgbtq  0.517181  straight  0.482819
2    The Hill      america     lgbtq  0.537367  straight  0.462633
  news_source compare_word wordlist1  d1/d1+d2 wordlist2  d2/d1+d2
0         NYT         love     lgbtq  0.531965  straight  0.468035
1         NYT       family     lgbtq  0.521382  straight  0.47

In [24]:
print_word_dists_each_model(['ocasio'], adjectives)

(423, 4)
       source    word           adj      dist
334  Fox News  ocasio   progressive  0.365418
16   Fox News  ocasio     sarcastic  0.465474
264  Fox News  ocasio  enthusiastic  0.471049
62   Fox News  ocasio    intolerant  0.496467
365  Fox News  ocasio       liberal  0.497042
(423, 4)
      source    word           adj      dist
137  Reuters  ocasio  discontented  0.388344
64   Reuters  ocasio      scornful  0.414708
365  Reuters  ocasio       liberal  0.426324
54   Reuters  ocasio    idealistic  0.427847
212  Reuters  ocasio      eloquent  0.430797
(423, 4)
       source    word          adj      dist
334  The Hill  ocasio  progressive  0.525036
365  The Hill  ocasio      liberal  0.542625
182  The Hill  ocasio   bewildered  0.556756
86   The Hill  ocasio  thoughtless  0.586852
133  The Hill  ocasio     dogmatic  0.591986
(423, 4)
    source    word           adj      dist
334    NYT  ocasio   progressive  0.304151
365    NYT  ocasio       liberal  0.445957
133    NYT  ocasio 

In [162]:
print_word_dists_each_model(['shooting'], adjectives)

(423, 4)
       source      word          adj      dist
179  Fox News  shooting  disciplined  0.486670
38   Fox News  shooting  frightening  0.510087
92   Fox News  shooting        stoic  0.525629
304  Fox News  shooting      hurried  0.540596
43   Fox News  shooting    tasteless  0.542874
(423, 4)
      source      word       adj      dist
92   Reuters  shooting     stoic  0.496251
187  Reuters  shooting  athletic  0.525407
64   Reuters  shooting  scornful  0.532825
268  Reuters  shooting    heroic  0.535264
59   Reuters  shooting    crafty  0.552180
(423, 4)
       source      word           adj      dist
115  The Hill  shooting         crisp  0.521371
149  The Hill  shooting  contemptible  0.606837
86   The Hill  shooting   thoughtless  0.609420
182  The Hill  shooting    bewildered  0.609676
304  The Hill  shooting       hurried  0.615771
(423, 4)
    source      word           adj      dist
239    NYT  shooting    disturbing  0.521737
117    NYT  shooting         naive  0.551564
3

In [29]:
print_word_dists_each_model(['metoo'])

(423, 4)
       source   word          adj      dist
131  Fox News  metoo       sordid  0.397127
81   Fox News  metoo     aspiring  0.448535
96   Fox News  metoo    outspoken  0.494659
257  Fox News  metoo  spontaneous  0.505243
75   Fox News  metoo     forceful  0.507450
(423, 4)
      source   word         adj      dist
93   Reuters  metoo  hysterical  0.410461
6    Reuters  metoo  effeminate  0.440745
62   Reuters  metoo  intolerant  0.446195
297  Reuters  metoo   emotional  0.467436
239  Reuters  metoo  disturbing  0.469760
(423, 4)
       source   word          adj      dist
257  The Hill  metoo  spontaneous  0.451610
200  The Hill  metoo  imaginative  0.477864
212  The Hill  metoo     eloquent  0.485439
133  The Hill  metoo     dogmatic  0.486279
93   The Hill  metoo   hysterical  0.489868
(423, 4)
    source   word           adj      dist
93     NYT  metoo    hysterical  0.494415
128    NYT  metoo   reactionary  0.513331
133    NYT  metoo      dogmatic  0.522175
64     NYT  meto

In [31]:
print_word_dists_each_model(['american'])

(423, 4)
       source      word            adj      dist
286  Fox News  american      oppressed  0.430025
143  Fox News  american      repressed  0.470680
203  Fox News  american  conscientious  0.476445
183  Fox News  american      courteous  0.496688
303  Fox News  american      agreeable  0.500179
(423, 4)
      source      word           adj      dist
182  Reuters  american    bewildered  0.555218
64   Reuters  american      scornful  0.565878
0    Reuters  american  disorganized  0.575307
20   Reuters  american      venomous  0.578273
187  Reuters  american      athletic  0.583312
(423, 4)
       source      word             adj      dist
44   The Hill  american       sedentary  0.452374
13   The Hill  american       resentful  0.462171
216  The Hill  american      benevolent  0.483714
399  The Hill  american        ordinary  0.485799
2    The Hill  american  impressionable  0.493461
(423, 4)
    source      word           adj      dist
137    NYT  american  discontented  0.48510